In [14]:
%matplotlib inline
import pandas as pd
import numpy as np
import geopandas as gpd
from lib.funcs import *

In [15]:
## Get the names and open data
data_domicilio_path = ('./data/CENSO/Domicilio02_PB.csv') 

with open(data_domicilio_path, "r", encoding="latin-1") as myfile:
    for i in range(1):
        line = next(myfile).strip()
        names = line.split(';')


data = pd.read_csv(data_domicilio_path,                  
                sep=';', 
                header=None, 
                names=names,
                skiprows=1,
                encoding="latin-1",
                dtype={'Cod_setor': str})

In [16]:
## Open SHP
setores = gpd.read_file('./data/shp/jp_shp.shp')
setores = setores.to_crs(epsg=3857)

In [17]:
border = setores
border['new_column'] = 0
border = border.dissolve(by='new_column')
border = border[['geometry']]
border['geometry'] = border['geometry'].buffer(0.0001)

In [18]:
# Select only the data we are interested in
#v001 = Moradores em domicilios particular e domicilios coletivos
moradores = data[['Cod_setor', 'V001']]
moradores = moradores.rename(columns={'V001': 'MORADORES'})

setores = setores[['geometry', 'NM_BAIRRO', 'CD_GEOCODI']]

In [19]:
# Set the same inde
moradores.set_index('Cod_setor', inplace=True)
setores.set_index('CD_GEOCODI', inplace=True)

In [20]:
moradores.to_csv('./data/POPULACAO/populacao_parse.csv')

In [21]:
setores = setores.join(moradores)

In [22]:
bairros = setores.dissolve(by='NM_BAIRRO', aggfunc='sum')

In [23]:
bairros_interesse = ['Torre', 'Tambauzinho', 'Expedicionários', 'Miramar', 'Pedro Gondim', 'Estados']
setores_recorte = setores.loc[setores['NM_BAIRRO'].isin(bairros_interesse)]

In [24]:
setores_recorte['area'] = setores_recorte.apply(get_area, axis=1)
setores_recorte['densidade'] = setores_recorte.apply(get_density, axis=1)
bairros['area'] = bairros.apply(get_area, axis=1)
bairros['densidade'] = bairros.apply(get_density, axis=1)

<ipython-input-24-6e7c8dbd7c66>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  setores_recorte['area'] = setores_recorte.apply(get_area, axis=1)
<ipython-input-24-6e7c8dbd7c66>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  setores_recorte['densidade'] = setores_recorte.apply(get_density, axis=1)


In [25]:
setores_recorte = setores_recorte.to_crs(epsg=3857)
setores_recorte.to_file('./data/POPULACAO/setores_recorte.shp')

bairros = bairros.to_crs(epsg=3857)
bairros.to_file('./data/POPULACAO/bairros.shp')

In [26]:
export(setores_recorte, 'populacao_setores_recorte', 3857)
export(bairros, 'populacao_bairros', 3857)